In [1]:
import pandas as pd
import json

from serlo.mysql import db

In [2]:
class entity:
    def __init__(self, id, empty_boxes):
        self.id = id
        self.empty_boxes = empty_boxes
        
    

In [3]:
#def get_items(obj):

def get_keys(obj, stack):
    #print("this: ")
    #print(obj)
    for k, v in obj.items():
      k2 = ([k] if k else []) + stack # don't return empty keys
      if v and isinstance(v, dict):
        for c in get_keys(v, k2):
          yield c
      else: # leaf
        yield k2

def get_values(obj):
    for v in obj.values():
      if not v: continue
      if isinstance(v, dict):
        for c in get_values(v):
          yield c
      else: # leaf
        yield v if isinstance(v, list) else [v]
        
def get_value_from_key_array(obj, keys):
    #print(keys)
    if len(keys) == 0:
        return
    if len(keys) == 1:
        return obj[keys[0]]
    else:
        key = keys.pop(-1)
        #print(key)
        return get_value_from_key_array(obj[key], keys)

  #return list(get_keys(obj,[])), list(get_values(obj))

In [9]:
def find_empty_boxes(obj):
    empty_boxes = []
    #print(obj)
    keys = list(get_keys(obj, []))
    #print("keys")
    #print(keys)
    for i in range(len(keys)):
        leaf = keys[i][0]
        if leaf == "plugin":
            plugin = {}
            if len(keys[i]) == 1:
                plugin = obj
            else:
                #print("test")
                plugin = get_value_from_key_array(obj, keys[i][1:])
            if plugin["plugin"] == "box":
                #print("box")
                #print(plugin)
                if is_empty_box(plugin["state"]):
                    empty_boxes.append(plugin["state"]["anchorId"])
                    #print(1)
        elif leaf == "state":
            #print(i)
            #print(keys[i])
            #print("value")
            #print(get_value_from_key_array(obj, keys[i]))
            state = get_value_from_key_array(obj, keys[i])
            if type(state) == list:
                for sub_obj in state:
                    if not sub_obj:
                        continue
                    #print("subobj: ")
                    #print(sub_obj)
                    #print("theses: ")
                    #print(find_empty_boxes(sub_obj))
                    empty_boxes.extend(find_empty_boxes(sub_obj))
    return empty_boxes
                    
def handle_boxes(boxes):
    if boxes:
        for box in boxes:
            if is_empty_box(box):
                handle_empty_box(box)
                    
                    
def is_empty_box(box):
    keys = list(get_keys(box, []))
    for i in range(len(keys)):
        leaf = keys[i][0]
        if leaf == "text":
            if get_value_from_key_array(box, keys[i]):
                return False
    return True
    
def is_empty_row(row):
    state = row["state"]
    match row["plugin"]:
        case "rows":
            for sub_row in row["state"]:
                if not is_empty_row(sub_row):
                    return False
        case "text":
            return empty_children(state)
        case "image":
            #TODO
            return is_empty_image(state)
        case "equations":
            #TODO
            return False
        case "multimedia":
            #TODO
            return is_empty_multimedia(state)
        case "serloTable":
            #TODO
            return False
        case "highlight":
            #TODO
            return False
        case _:
            raise Exception('Unhandled plugin: {}'.format(row))
    return True

def is_empty_image(image):
    src = image["src"]
    return not src or src.isspace()

def is_empty_multimedia(multimedia):
    if is_empty_row(multimedia["multimedia"]):
        print("empty multimedia")
    return is_empty_row(multimedia["explanation"]) and is_empty_row(multimedia["multimedia"])

def is_empty_box(box):
    #print(box)
    content = box["content"]
    if content["plugin"] != "rows":
        raise Exception('The box content is invalid: {}'.format(content))
        
    for row in content["state"]:
        if not is_empty_row(row):
            return False
    return True

#not needed
def is_empty_object(obj):
    #print("plugin")
    #print(obj)
    match obj["plugin"]:
        case "rows":
            for row in obj["state"]:
                if not is_empty_object(row):
                    return False
        case "text":
            for text in obj["state"]:
                if text["type"] == 'p':
                    return is_empty_text(text["children"])
                else:
                    print("unhandled text type: ")
                    print(text["type"])
                    return False
        case _:
            print("unhandled plugin: ")
            print(obj["plugin"])
            return False
            
    return True
    
#not needed
def is_empty_text(text):
    #print("children")
    #print(text)
    for section in text:
        if not empty_children(section["children"]):
            return False
    return True

def empty_children(children):
    for child in children:
        if not child:
            continue
        elif "text" in child:
            if child["text"] and not child["text"].isspace():
                return False
        elif "src" in child:
            if child["type"] != "math":
                print("not math")
                print(child)
            if child["src"] and not child["src"].isspace():
                return False
        elif "children" in child:
            if not empty_children(child["children"]):
                return False
        else:
            raise Exception('Unhandled text: {}'.format(child))
    return True

In [5]:
df = pd.read_sql("""
SELECT entity.id, value
FROM entity_revision_field, entity
where field = "content"
    AND entity.current_revision_id = entity_revision_field.entity_revision_id;
""", db)

df.head()

/home/moe/.asdf/installs/python/3.10.1/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,value
0,1495,"{""plugin"":""article"",""state"":{""introduction"":{""..."
1,1497,"{""plugin"":""article"",""state"":{""introduction"":{""..."
2,1499,"{""plugin"":""article"",""state"":{""introduction"":{""..."
3,1501,"{""plugin"":""article"",""state"":{""introduction"":{""..."
4,1503,"{""plugin"":""article"",""state"":{""introduction"":{""..."


In [10]:
entities = []
for i, revision in df.iterrows():
    #print("id")
    #print(revision["id"])
    try:
        currentObject = json.loads(revision["value"])
    except:
        #print("id")
        continue
    if isinstance(currentObject, dict):
        empty_boxes = find_empty_boxes(currentObject)
        if len(empty_boxes):
            entities.append(entity(revision["id"], empty_boxes))
            

In [11]:
for entity in entities:
    print(entity.id)
    print (entity.empty_boxes)

1769
['box75966']
60274
['box21315']
148316
['box31707']
149487
['box66070']
192951
['box48647']
195446
['box65044']
223711
['box35853']
240351
['box85162', 'box76211', 'box38350']


In [ ]:
test = pd.read_sql("""
SELECT entity.id, value
FROM entity_revision_field, entity
where field = "content"
    AND entity.current_revision_id = entity_revision_field.entity_revision_id
    AND entity.id = 5024;
""", db)

test

In [ ]:
entities[0].empty_boxes

In [ ]:
bytes(df["value"][400], "latin1").decode("utf8")

In [ ]:
test = '{"plugin":"article","state":{"introduction":{"plugin":"articleIntroduction","state":{"explanation":{"plugin":"text","state":[{"type":"p","children":[{"text":"In diesem Artikel werden Boxen und ihre Verwandten ausprobiert :)"}]}]},"multimedia":{"plugin":"image","state":{"src":"","caption":{"plugin":"text","state":[{"type":"p","children":[{}]}]}}},"illustrating":true,"width":50}},"content":{"plugin":"rows","state":[{"plugin":"text","state":[{"type":"p","children":[{"text":"Text Anfang"}]}]},{"plugin":"box","state":{"title":{"plugin":"text","state":[{"type":"p","children":[{}]}]},"content":{"plugin":"rows","state":[{"plugin":"text","state":[{"type":"p","children":[{"text":"Das hier ist ein Merksatz. In der Plugin-Beschreibung steht auch, sodass dieser bisher wohl nicht besonders semantisch verwendet wurde."}]},{"type":"p","children":[{"text":""}]}]}]},"type":"blank","anchorId":"box80970"}},{"plugin":"text","state":[{"type":"p","children":[{"text":"Zwischentext zwischen den Elementen"}]}]},{"plugin":"box","state":{"title":{"plugin":"text","state":[{"type":"p","children":[{}]}]},"content":{"plugin":"rows","state":[{"plugin":"text","state":[{"type":"p","children":[{"text":"Das hier ist ein Zitat. Angeblich ist es einfach eingerückt, oder passiert hier noch mehr?"}]},{"type":"p","children":[{"text":""}]}]}]},"type":"quote","anchorId":"box35521"}},{"plugin":"text","state":[{"type":"p","children":[{"text":"Noch ein schicker Zwischentext"}]}]},{"plugin":"spoiler","state":{"title":"Das ist der Spoiler-Titel","content":{"plugin":"rows","state":[{"plugin":"text","state":[{"type":"p","children":[{"text":"Das ist der Spoiler-Inhalt, hoffentlich."}]}]}]}}},{"plugin":"text","state":[{"type":"p","children":[{"text":"Und zum Abschluss kommt noch mal normaler Text"}]}]},{"plugin":"text","state":[{"type":"p","children":[{"text":"Und hier ein Experiment mit Tastendrücken: "},{"type":"math","src":"\\\\fbox {~1~} ~\\\\fbox{Enter}~\\\\fbox{~2~} ~\\\\fbox{Enter}~\\\\fbox{~+~}","inline":true,"children":[{"text":"1 Enter"}]},{"text":""}]}]},{"plugin":"text","state":[{"type":"p","children":[{}]}]},{"plugin":"box","state":{"type":"note","title":{"plugin":"text","state":[{"type":"p","children":[{}]}]},"anchorId":"box32917","content":{"plugin":"rows","state":[{"plugin":"text","state":[{"type":"p","children":[{"text":"Das ist eine schicke neue Box."}]}]}]}}}]},"exercises":[],"exerciseFolder":{"id":"","title":""},"relatedContent":{"articles":[],"courses":[],"videos":[]},"sources":[]}}'

In [ ]:
obj = json.loads(test)
keys = list(get_keys(obj, []))
keys

In [ ]:
True

In [ ]:
test = {
  "plugin": "box",
  "state": {
    "type": "remember",
    "title": {
      "plugin": "text",
      "state": [
        {
          "type": "p",
          "children": [
            {
              "text": "Warum ist die Quadratwurzel immer eine positive Zahl?"
            }
          ]
        }
      ]
    },
    "anchorId": "box60327",
    "content": {
      "plugin": "rows",
      "state": [
        {
          "plugin": "text",
          "state": [
            {
              "type": "p",
              "children": [
                {
                  "text": "Betrachtet man das Beispiel "
                },
                {
                  "type": "math",
                  "src": "\\sqrt4",
                  "inline": "True",
                  "children": [
                    {
                      "text": "\\sqrt4"
                    }
                  ]
                },
                {
                  "text": " , dann ist die Quadratwurzel diejenige Zahl, welche man quadrieren muss um 4 zu erhalten. Offensichtlich lÃ¶st 2 das Problem. Aber auch -2 wÃ¤re prinzipiell eine sinnvolle LÃ¶sung, denn "
                },
                {
                  "type": "math",
                  "src": "(-2)\\cdot(-2)=(-2)^2=4",
                  "inline": "True",
                  "children": [
                    {
                      "text": "(-2)\\cdot(-2)=(-2)^2=4"
                    }
                  ]
                },
                {
                  "text": ".\nWarum ist also (-2) nicht die Quadratwurzel von 4?\nDies liegt daran, dass die Quadratwurzel auch als "
                },
                {
                  "type": "a",
                  "href": "/1583",
                  "children": [
                    {
                      "text": "Funktion"
                    }
                  ]
                },
                {
                  "text": " definiert werden kann bzw. soll. Eine Funktion ordnet jedem Wert aus dem "
                },
                {
                  "type": "a",
                  "href": "/1575",
                  "children": [
                    {
                      "text": "Definitionsbereich"
                    }
                  ]
                },
                {
                  "text": " "
                },
                {
                  "text": "genau eine",
                  "strong": "True"
                },
                {
                  "text": " Zahl aus der "
                },
                {
                  "type": "a",
                  "href": "/1655",
                  "children": [
                    {
                      "text": "Wertemenge"
                    }
                  ]
                },
                {
                  "text": " zu. Deshalb definert man die Quadratwurzel immer als eine positive Zahl (oder 0), sodass die Quadratwurzel "
                },
                {
                  "text": "eindeutig",
                  "strong": "True"
                },
                {
                  "text": " ist.  "
                }
              ]
            },
            {
              "type": "p",
              "children": [
                {
                  "text": "Die Quadratwurzel einer Zahl soll immer positiv sein, weil man einen eindeutigen Ausdruck haben mÃ¶chte."
                }
              ]
            }
          ]
        }
      ]
    }
  }
}

if "plugwsefin" in test and test["siotrjo"]:
    print("what")
else:
    print("aeporgk")